In [1]:
import torch
import torch.nn as nn

In [2]:
# model_path = 'model/FMTopK/FMTopK__MovieLens_1M/ML_1MTOPK/_context101__0__lr=0.001__l2=0.0__emb_size=64__loss_n=BPR.pt'
# model_path = "model/LightGCN/LightGCN__MovieLens_1M/ML_1MTOPK/__0__lr=0.001__l2=1e-08__emb_size=64__n_layers=3__batch_size=256.pt"
model_path = "D:/ML/recommendation/ReChorus/model/SAMTopK/SAMTopK__MovieLens_10M/ML_10MTOPK/_context101__0__lr=0.001__l2=0.0001__emb_size=64__loss_n=BPR__interaction_type=SAM3A__aggregation=mean_pooling.pt"
model = torch.load(model_path)


print(type(model))
# model.eval()

<class 'collections.OrderedDict'>


In [3]:
print(model)

OrderedDict([('block.layers.0.W', tensor([[[-1.2722e-04, -1.5826e-03,  5.2965e-04,  ..., -4.6991e-05,
           2.1312e-06,  2.7775e-03],
         [ 1.5885e-04,  1.6898e-03,  8.5460e-04,  ..., -1.5946e-04,
          -2.2485e-03,  5.1306e-04],
         [-2.1956e-03, -1.1348e-02,  2.3028e-02,  ...,  1.9875e-02,
          -1.2839e-02, -3.5946e-03],
         ...,
         [ 3.0502e-03,  2.0143e-02, -1.0948e-02,  ..., -4.1185e-04,
          -3.5502e-03, -1.1238e-02],
         [ 1.2137e-05, -3.7069e-05, -2.2113e-04,  ..., -2.9981e-05,
           3.4273e-03, -6.1671e-05],
         [-1.0422e-04, -4.0202e-03, -3.2837e-05,  ..., -4.1395e-04,
          -1.3681e-03,  3.0752e-03]],

        [[ 2.1654e-04,  6.1120e-04, -3.4341e-04,  ..., -3.0025e-04,
          -2.1394e-03,  7.2628e-04],
         [ 3.5713e-04, -1.8808e-04,  4.7847e-04,  ..., -6.1742e-04,
          -3.3318e-03,  2.2635e-04],
         [ 1.3525e-03, -8.9233e-03,  1.9785e-02,  ..., -7.3070e-03,
          -1.4584e-02, -2.4440e-02],
     

In [4]:
for key, _ in model.items():
    print(key)

block.layers.0.W
block.layers.0.K.weight
block.layers.0.Q.weight
embedding_dict.i_genre_c.weight
embedding_dict.i_title_c.weight
embedding_dict.c_day_f.weight
embedding_dict.c_hour_c.weight
embedding_dict.c_period_c.weight
embedding_dict.c_weekday_c.weight
embedding_dict.user_id.weight
embedding_dict.item_id.weight
output_layer.weight
output_layer.bias


In [5]:
for key, value in model.items():
    print(key, len(value))

block.layers.0.W 8
block.layers.0.K.weight 64
block.layers.0.Q.weight 64
embedding_dict.i_genre_c.weight 651
embedding_dict.i_title_c.weight 7699
embedding_dict.c_day_f.weight 64
embedding_dict.c_hour_c.weight 24
embedding_dict.c_period_c.weight 9
embedding_dict.c_weekday_c.weight 7
embedding_dict.user_id.weight 58668
embedding_dict.item_id.weight 7699
output_layer.weight 1
output_layer.bias 1


In [6]:
def convert_rechorus_to_agent4rec(rechorus_path, save_path, target_user_dim=1000, target_item_dim=3292, epoch=0):
    # 加载ReChorus模型
    rechorus_model = torch.load(rechorus_path)
    
    # 创建新的state_dict格式
    new_state_dict = {
        'epoch': epoch,
        'state_dict': {}
    }

    # 获取用户和物品embedding的key，需要根据具体模型的key进行调整
    # userid= 'encoder.embedding_dict.user_emb'
    # itemid= 'encoder.embedding_dict.item_emb'
    userid = 'embedding_dict.user_id.weight'
    itemid = 'embedding_dict.item_id.weight'
    
    # 转换并调整用户embedding维度
    if userid in rechorus_model:
        user_weights = rechorus_model[userid]
        if user_weights.size(0) > target_user_dim:
            new_state_dict['state_dict']['embed_user.weight'] = user_weights[:target_user_dim]
        else:
            # 如果目标维度更大，需要随机初始化额外的embedding
            new_weights = torch.randn(target_user_dim, user_weights.size(1))
            new_weights[:user_weights.size(0)] = user_weights
            new_state_dict['state_dict']['embed_user.weight'] = new_weights
            
    # 转换并调整物品embedding维度
    if itemid in rechorus_model:
        item_weights = rechorus_model[itemid]
        if item_weights.size(0) > target_item_dim:
            new_state_dict['state_dict']['embed_item.weight'] = item_weights[:target_item_dim]
        else:
            # 如果目标维度更大，需要随机初始化额外的embedding
            new_weights = torch.randn(target_item_dim, item_weights.size(1))
            new_weights[:item_weights.size(0)] = item_weights
            new_state_dict['state_dict']['embed_item.weight'] = new_weights
            
    # 保存转换后的模型
    torch.save(new_state_dict, save_path)

In [7]:
save_path = '../converted_models/rechorus_test_model_2.checkpoint.pth.tar'

convert_rechorus_to_agent4rec(model_path, save_path, epoch=24)

In [ ]:
def convert_rechorus_to_agent4rec(rechorus_path, save_path, config_path=None, target_dims=None):
    """
    转换ReChorus模型到Agent4Rec格式
    
    Args:
        rechorus_path: ReChorus模型路径
        save_path: 保存转换后的模型路径
        config_path: 保存配置文件的路径
        target_dims: 目标维度字典 {'user': user_dim, 'item': item_dim}
    """
    # 加载ReChorus模型
    rechorus_model = torch.load(rechorus_path)
    
    # 确定模型类型
    if 'encoder.embedding_dict' in str(rechorus_model):
        model_type = 'LightGCN'
        user_key = 'encoder.embedding_dict.user_emb'
        item_key = 'encoder.embedding_dict.item_emb'
    else:
        model_type = 'FM'
        user_key = 'context_embedding.user_id.weight'
        item_key = 'context_embedding.item_id.weight'
    
    # 创建新的state_dict格式
    new_state_dict = {
        'epoch': 0,
        'state_dict': {}
    }
    
    # 转换embedding层
    for old_key, new_key, dim_key in [
        (user_key, 'embed_user.weight', 'user'),
        (item_key, 'embed_item.weight', 'item')
    ]:
        if old_key in rechorus_model:
            weights = rechorus_model[old_key]
            if target_dims and dim_key in target_dims:
                target_dim = target_dims[dim_key]
                if weights.size(0) > target_dim:
                    weights = weights[:target_dim]
                elif weights.size(0) < target_dim:
                    new_weights = torch.randn(target_dim, weights.size(1))
                    new_weights[:weights.size(0)] = weights
                    weights = new_weights
            new_state_dict['state_dict'][new_key] = weights
    
    # 保存转换后的模型
    torch.save(new_state_dict, save_path)
    
    # 生成配置文件
    if config_path:
        config = {
            "vis": -1,
            "seed": 0,
            "clear_checkpoints": True,
            "candidate": False,
            "test_only": False,
            "data_path": "../data/",
            "dataset": "ml-1m",
            "embed_size": 64,
            "batch_size": 256,
            "lr": 0.001,
            "regs": 1e-8,
            "epoch": 200,
            "Ks": "5,10,20,50",
            "verbose": 5,
            "saveID": model_type.lower(),
            "patience": 10,
            "checkpoint": "./",
            "modeltype": model_type,
            "cuda": 0,
            "n_layers": 3 if model_type == 'LightGCN' else 0,
            "neg_sample": 1,
            "num_workers": 0,
            "train_norm": False,
            "pred_norm": False,
            "nodrop": True
        }
        
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=2)

# 使用示例
if __name__ == "__main__":
    rechorus_path = "model/LightGCN/model.pt"
    save_path = "converted_models/agent4rec_model.pth.tar"
    config_path = "converted_models/config.json"
    target_dims = {'user': 1000, 'item': 3292}
    
    convert_rechorus_to_agent4rec(
        rechorus_path=rechorus_path,
        save_path=save_path,
        config_path=config_path,
        target_dims=target_dims
    )